The goal of this notebook is to 

First we need to set up a number of variables we going to use later in the notebook:
- OPENAI_API_KEY: to access the OpenAI embeddings (secret)
- AZURESEARCH_ADMIN_KEY: to access the Azure Search service (secret)
- AZURESEARCH_ENDPOINT (theoretically it's not a secret, but it's better to keep it in .env file)
- AZURESEARCH_INDEX_NAME (theoretically it's not a secret, but it's better to keep it in .env file)
- if you create index in Azure Portal (not programatically), the default field names will be different than the ones assumed by the AzureSearch in langchain_community package. You can set the following variables to match the field names in your index:
    - AZURESEARCH_FIELDS_CONTENT_VECTOR=text_vector (it's not a secret) 
    - AZURESEARCH_FIELDS_ID=chunk_id (it's not a secret)
    - AZURESEARCH_FIELDS_CONTENT=chunk (it's not a secret)
    
All above will be read from .env file.

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv

In [ ]:
_ = load_dotenv(find_dotenv(filename='.env'))

The first thing we need is to build a retriever. 


We will use Microsoft AI Search with hybrid mode to retrieve the most relevant documents for a given query.

Source code for langchain_community.vectorstores.azuresearch: 
https://api.python.langchain.com/en/latest/_modules/langchain_community/vectorstores/azuresearch.html#AzureSearch.similarity_search

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores.azuresearch import AzureSearch

# OpenAI API data (for embeddings)
openai_api_key = os.getenv("OPENAI_API_KEY")
openai_api_version = "2023-05-15"
model = "text-embedding-ada-002"

# Azure AI Search data (for vector store)
vector_store_address = os.getenv("AZURESEARCH_ENDPOINT") 
vector_store_password = os.getenv("AZURESEARCH_ADMIN_KEY")
vector_store_index = os.getenv("AZURESEARCH_INDEX_NAME")

# Initialize the embeddings
embeddings = OpenAIEmbeddings(
    openai_api_key=openai_api_key, 
    openai_api_version=openai_api_version, 
    model=model
)

# Alternatively, we may use AzureOpenAIEmbeddings
# from langchain_openai import AzureOpenAIEmbeddings
# embeddings = AzureOpenAIEmbeddings(
#     model="<embeding-model>",
#     azure_endpoint="<Azure_openai_endpoint>",
#     azure_ad_token_provider=token_provider
# )

# Initialize the vector store
vector_store = AzureSearch(
    azure_search_endpoint=vector_store_address,
    azure_search_key=vector_store_password,
    index_name=vector_store_index,
    embedding_function=embeddings.embed_query,
)

In [ ]:
# There multiple way of seting up the retriever, the issue here is to initialize it this way that is uses hybrid not only similarity search
# I use this one: https://stackoverflow.com/questions/78576496/hybrid-search-using-azure-ai-search-and-lang-chain-as-a-retriever
retriever = vector_store.as_retriever(k=5, search_type="hybrid")

# There is also Azure AI Search retriever https://python.langchain.com/v0.2/docs/integrations/retrievers/azure_ai_search/
# Here is the doc: https://python.langchain.com/v0.2/docs/integrations/retrievers/azure_ai_search/

# Another way discussed here: https://github.com/langchain-ai/langchain/discussions/18752

In [ ]:
# Some example queries
# query = "What are the fully local agents?"
# query = "What is Nvidia NIM API?"
query = "What is a generator function?"
docs = retriever.invoke(query)
for doc in docs:
    print(doc.page_content)

In [ ]:
# imports required for chains
from langchain_ollama import ChatOllama
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser

In [ ]:
# We define two versions of the chain - one with Ollama (llama3.1) and one with OpenAI (gpt-4o-mini)
# PROVIDER = "ollama"
PROVIDER = "openai"

In [ ]:
# The main chain - it will use retrieved documents to generate an answer to the user question
prompt = PromptTemplate(
    template="""You are an assistant for question-answering tasks. 
    
    Analyze carefully and use the following documents to answer the user question. 
    
    If you don't know the answer, just say that you don't know. Do not make up an answer. 
    
    Keep your answer short and to the point.
    
    Question: {question} 
    Documents: {documents} 
    Answer: 
    """,
    input_variables=["question", "documents"],
)

if PROVIDER == "openai":
    llm = ChatOpenAI(
        model="gpt-4o-mini",
        temperature=0,
    )
else:
    llm = ChatOllama(
        model="llama3.1",
        temperature=0,
    )

main_chain = prompt | llm | StrOutputParser()

In [ ]:
# Test the main chain
main_chain.invoke({"question": "What's my city?", "documents": "I'm based in Kuala Lumpur"})

In [ ]:
# Let's define a evaluator chain - it will evaluate if information retrieved by RAG retriver 
# is sufficient to answer the question. It returns yes or no in a JSON format.
eval_prompt = PromptTemplate(
    template="""Your task is to carefully evaluate if information in Documents provided below is sufficient for answering User Question.  
    User Question: {question} 
    Documents: {documents} 

    Return your Evaluation in JSON format with a single key 'Evaluation' and binary score 'yes' or 'no'. No other keys, values or preambles are allowed.
    
    If you don't know the answer pick 'no'.
    
    Evaluation: 
    """,
    input_variables=["question", "documents"],
)

if PROVIDER == "openai":
    eval_llm = ChatOpenAI(
        model="gpt-4o-mini",
        temperature=0,
        model_kwargs={ "response_format": { "type": "json_object" } },
    )
else:
    eval_llm = ChatOllama(
        model="llama3.1",
        temperature=0,
        format="json",
    )

eval_chain = eval_prompt | eval_llm | JsonOutputParser()

In [ ]:
# Test the evaluator chain - positive case
eval_chain.invoke({"question": "What's my city?", "documents": "I'm based in Kuala Lumpur"})

In [ ]:
# Test the evaluator chain - negative case
eval_chain.invoke({"question": "What's my city?", "documents": "US presidential elections are in November"})

In [ ]:
# Define websearch tool - we use Tavilyn for this
from langchain.schema import Document
from langchain_community.tools.tavily_search import TavilySearchResults

web_search_tool = TavilySearchResults()

In order to use LangGraph, we need to define GraphState class. This class persists the state of the graph or in other words, it stores a session.

In [ ]:
from typing_extensions import TypedDict, List
from langgraph.graph import START, END, StateGraph

class GraphState(TypedDict):
    """
    Represents the state of the graph.

    Attributes:
        question: question
        documents: list of retrieved documents
        answer: LLM generated answer
        search_required: whether to search web
        search_results: results of web search
        steps: steps of the graph execution
        
    """

    question: str
    documents: List[str]
    answer: str
    search_required: bool
    search_results: List[str]
    steps: List[str]


In the sections below we will create a series of functions that will serve as nodes in the graph: retrieve, evaluate, generate and search-web.

In [ ]:
# retrieves documents using previously defined retriever.
# consumes a state with a question
# returns a new state with documents added and appended step
def retrieve(state):
    question = state["question"]

    documents = retriever.invoke(question)
    
    steps = state["steps"]
    steps.append("retrieve_documents")
    
    return {
        "documents": documents, 
        "question": question, 
        "steps": steps
    }


In [ ]:
# generates an answer using previously defined main_chain
# consumes a state with a question and documents
# returns a new state with answer added and appended step
def generate(state):
    question = state["question"]
    documents = state["documents"]
    search_required = state["search_required"]
    search_results = state["search_results"]
    
    # if search was required and results are available, we use them instead of retrieved documents
    if search_required and search_results:
        documents = search_results

    answer = main_chain.invoke({"documents": documents, "question": question})

    steps = state["steps"]
    steps.append("generate_answer")
    
    return {
        "documents": documents,
        "question": question,
        "answer": answer,
        "search_required": search_required,
        "search_results": search_results,
        "steps": steps,
    }

In [ ]:
# evaluates if the documents are relevant to the question
# consumes a state with a question and documents
# returns a new state with search_required added and appended step
def evaluate(state):
    question = state["question"]
    documents = state["documents"]
    steps = state["steps"]
    steps.append("evaluate_retrieval")

    # Evaluate if the documents are relevant to the question
    evaluation = eval_chain.invoke({"documents": documents, "question": question})

    # if the evaluation is negative we set search_required to True
    search_required = False
    search_required = evaluation["Evaluation"] == "no"

    return {
        "documents": documents,
        "question": question,
        "search_required": search_required,
        "steps": steps,
    }

In [ ]:
# searches the web for documents that may help to answer the question
# consumes the question 
# produces the search results
def web_search(state):
    question = state["question"]
    documents = state["documents"]
    search_required = state["search_required"]
    steps = state["steps"]
    steps.append("web_search")
    
    results = web_search_tool.invoke({"query": question})

    search_results = [
        Document(page_content=doc["content"], metadata={"url": doc["url"]})
        for doc in results
    ]

    return {
        "documents": documents, 
        "question": question,
        "search_required": search_required,
        "search_results": search_results,
        "steps": steps
    }